# 预备知识

### tf.where(条件语句，A(真)，B(假))

    a = tf.constant([1,2,3,1,1])
    b = tf.constant([0,1,3,4,5])

### tf.greater()比较对应元素 (返回True或False的列表)

    c = tf.where(tf.greater(a,b),a,b)
    # 若a>b，返回a<b>对应位置</b>的元素，否则返回b<b>对应位置</b>的元素
    【out:】 tf.Tensor([1 2 3 4 5],shape=(5),dtype=int32)
    
### tf.random.RandomState.rand(维度)

    返回一个[0,1)之间的随机数（ 维度为空、返回张量）
    rdm = np.random.RandomState(seed=1)#seed常数每次生成的随机数相同

    a = rdm.rand() #返回一个随机标量
    b = rdm.rand(2,3)#返回一个2行3列的随机数矩阵
    print("a",a)
    print("b",b)
    【out:】
    a:0.417022...........
    b:[[7.20  .. ...],[.. .. ..]]

### np.vstack()

    将两个数组按<b>垂直方向</b>叠加

    a = np.array([1,2,3])
    b = np.array([4,5,6])
    c = np.vstack((a,b))
    print("c:",c)
    【out:】
    c:
    [[1 2 3],
    [4 5 6]]


### np.mgrid[]  & np.ravel  & np.c_[] 生成网格坐标点
    
    # 返回若干组维度相同的等差数组 [)
    np.mgrid[起始值：结束值：步长，起始值：结束值：步长,.....] 
    
    
    #把x变成一维数组
    np.ravel()
    
    # 使返回的间隔数值点配对输出
    np.c_[数组1，数组2，...]    

    x,y = np.mgrid[1:3:1,2:4:0.5]
    grid = np.c_[x.ravel(),y.ravel()]
    print('x',x)
    print('y',y)
    print('grid:\n',grid)
    【out:】
    x = [[1. 1. 1. 1.]
        [2. 2. 2. 2.]]
    y = [[2. 2.5 3. 3.5]
        [2. 2.5 3. 3.5]]
    grid:（拉直配对）
    [[1.2.]
    [1. 2.5]
    [1.3.]
    [1.3.5]
    [2.2.]
    [2.2.5]
    [2. 3.]
    [2.3.5]]
    

------------
# 常见函数

## 指数衰减学习率

指数衰减学习率 = 初始学习率 * 学习率衰减率 ** （当前轮数 / 多少轮衰减一次）

## sigmoid函数  

特点：
+ 映射到 0~1
+ 常用于选择工具
+ 易造成梯度消失
+ 输出非0均值，收敛慢
+ 幂运算复杂，训练时间长

## tanh函数 

+ 0均值 
+ 常用于转换工具

## Relu 

+ 最常用的激活函数
+ 收敛速度远大于上面两者

## 对初学者的建议：
+ 首选Relu激活函数
+ 学习率设置较小值
+ 输入特征标准化： 即让输入特征满足以0为均值，1为标准差的正态分布
+ 初始参数中心化：即让随机生成的参数满足以0为均值，$\sqrt{{2}\over{当前层输入特征个数}}$为标准差的正态分布

---------
# 损失函数

##### 预测值y与已知答案y_之间的差距

## 1.均方误差mse （Mean Squared Errot）
$${MSE(y_,y) }={ \sum{(y-y\_ )}^2 \over n}$$

    loss_mse = tf.reduce_mean(tf.square(y_ - y))
    
当使用均方误差mse，默认认为预测多了和预测少了损失是一样的，但是实际上，预测多了损失的可能是成本，预测少了损失的可能是利润。利润和成本往往不相等（参见2.自定义）

## 2.自定义

如预测酸奶销量、酸奶成本COST1元，利润PROFIT99元。预测少了损失利润99元，预测少了损失成本1元。所以希望生成的预测函数往多了预测

    loss_zdy = tf.reduce_sum(tf where :((tf.greater(y,y_),cost(y-y_),profit(y_-y)))

## 3.交叉熵ce (Cross Entropy)

交叉熵表示两个概率分布之间的距离，**交叉熵越大，两个概率分布越远。**

##### eg.二分类：

已知答案y_ = (1,0)表示第一种情况发生的概率为1，第二种情况发生的概率0，
预测y1 = (0.6,0.4) y2 = (0.8,0.2)哪个更接近标准答案y_?

    答：H1((1,0),(0.6,0.4)) = 0.511
    H2(...,...) = 0.223
    H1 > H2
    所以y2预测更准
    
    loss_ce1 = tf.losses.categorical_crossentropy([1,0],[0.6,0.4])

# softmax与交叉熵结合

输出先经过softmax函数，再计算y与y_的交叉熵损失函数

    tf.nn.softmax_cross_entropy_with_logits(y_,y)`
    # y_是正确答案
    y_ = np..array([[1 0 0],[0 1 0],[0 0 1],[1 0 0],[0 1 0]])
     
    y = np.array([[12, 3, 2], [3, 10, 1], [1, 2, 5], [4, 6.5, 1.2], [3, 6, 1]])     
    y_pro = tf.nn.softnax(y)
    
    # 分布计算
    loss_ce1 = tf.losses.categorical_crossentropy(y_,y_pro)
    # 等价于综合计算
    loss_ce2 = tf.nn.softmax_cross_entropy_with_logits(y_,y)
    # 计算结果loss_ce1 = loss_ce2

# 参数优化器

待优化参数w、损失函数loss、学习率lr、每迭代一个batch，t表示当前batch迭代的总次数
1. 计算t时刻损失函数关于当前参数的梯度${g_t} = {Δloss} $=   ${əloss} \over əw_t$
2. 计算t时刻一阶动量$m_t$和二阶动量$V_t$
    + 一阶动量：与梯度相关的函数
    + 二阶动量：与梯度平方相关的函数
3. 计算t时刻下降梯度：$ŋ_t = lr * m_t / \sqrt{V_t}$(学习率乘以一阶动量除以根号下二阶动量)

4. 计算t+1时刻参数：$W_{t+1} = W_t - ŋ_t = W_t - lr * m_t / \sqrt{V_t}$（当前时刻的参数减去学习率✖）

## SGD 无momentum，常用的梯度下降
$W_{t+1} = $ $W_t - lr * $ $əloss \over əw_t^*$

    w1.assign_sub(lr * grads[0])
    b1.assign_sub(lr * grads[1])

## SGDM 含有momentum的SGD，在SGD基础增加一阶动量
$m_t = β * m_{t-1} + (1 - β) * g_t$

而且上一时刻的一阶动量$m_{t-1}$占大头，因为β经验值接近0.9
二阶动量在SGDM中仍是恒等于1的 $V_t = 1$

把一阶动量和二阶动量带入ŋ的计算公式，

$ŋ_t = lr * m_t / \sqrt{V_t}$ = $lr * m_t$ = $ lr * (β * m_{t-1} + (1-β) * g_t)$

再把ŋ带入参数更新公式就可以用SGDM更新参数了

$W_{t+1} = W_t - ŋ_t$ = $ W_t - lr * (β * m_{t-1} + (1 - β) * g_t)$

用python实现的最重要的一步是把一阶动量和二阶动量计算出来

$m_t = β * m_{t-1} + (1 - β) * g_t$

$g_t$是当前时刻的梯度

    m_w, m_b = 0,0 # 第0时刻的一阶动量
    beta = 0.9
    
    # sgd-momentum
    m_w = beta * m_w + (1 - beta) * grads[0]
    m_b = beta * m_b + (1 - beta) * grads[1]
    w1.assign_sub(lr * m_w)
    b1.assign_sub(lr * m_b)


# Adagrad，在SGD的基础上增加二阶动量
`可以对模型中的每个参数分配自适应学习率了`

adagrad的一阶动量和SGD一样，是当前的梯度
$m_t = g_t$

二阶动量是从开始到现在、梯度平方的累加和
$V_t = \sum^{t}_{t=1} g_t^2$

有了一阶动量和二阶动量，可以带入参数更新参数了
$ŋ_t = lr * m_t / (\sqrt{V_t})$  = $lr * g_t / (\sqrt{\sum g_t^2})$

$W_{t+1} = W_t - ŋ_t = W_t - lr * g_t / (\sqrt{\sum g_t^2})$

adagrad的一阶动量mt就是当前时刻的梯度
$m_t = g_t  V_t = \sum^{t}_{t=1} g_t^2$

所以直接带入参数更新公式
设0时刻 w和b的二阶动量初始值是0

    v_w,v_b = 0,0
    # adagrad
    v_w += tf.square(grads[0])# 梯度平方累计和
    v_b += tf.square(grads[1])
    w1.assign_sub(lr * grads[0] / tf.sqrt(v_w))
    b1_assign_sub(lr * grads[1] / tf.sqrt(v_b))


# RMSProp是在SGD基础上增加二阶动量

二阶动量v使用煮熟滑动平均值计算，表征的是过去一段时间的平均值

同样，求出一阶动量和二阶动量后

$m_t = g_t$
$V_t = β * V_{t-1} + (1 - β) * g^2_t$

代入参数更新公式
$ŋ_t = lr * m_t / (\sqrt{V_t})$  = $ lr * g_t / (\sqrt{β * V_{t-1} + (1 - β) * g_t^2} )$

$W_{t+1} = W_t - ŋ_t$ = $w_t - lr * g_t /(\sqrt{β * V_{t-1} + (1 - β) * g_t^2} ) $

实现参数自更新

一阶动量$m_t$是梯度

$M_t = g_t V_t = β * V_{t-1} + (1 - β) * g^2_t$
    
    v_w,v_b = 0,0
    beta = 0.9
    v_w = beta * v_w + (1 - beta) * tf.square(grads[0])
    v_b = beta * v_b + (1 - beta) * tf.square(grads[1])
    w1.assign_sub(lr * grads[0] / tf.sqrt(v_w)
    b1.assign_sub(lr * grads[1] / tf.sqrt(v_b)

# adam 

同时结合SGDM一阶动量和RMSProp二阶动量，并在此基础上增加了`两个修正项`
$m_t = β_1 * m_{t-1} + (1 - β_1) * g_t$

修正一阶动量的偏差：$m_t $=$ m_t \over {1 - β_1^t}$

$V_t = β_2 * V_{step-1} + (1 - β_2) * g_t^2$

修正二阶动量的偏差：$V_t = V_t \over (1 - β_2^t)$

$ŋ_t = lr * m_t / \sqrt{V_t }$ 

$W_{t+1} = W_t - ŋ_t$

    m_w，m_b = 0,0
    v_w,v_b = 0,0
    beta1, beta2 = 0.9,0.999
    delta_w, delta_b = 0,0
    global_step = 0
    # global_step 是训练开始到当前时刻所经历的总batch数
    
    #adam一阶动量表达式和含momentum的sgd一阶动量表达式是相同的
    m_w = beta1 * m_w + (1 - beta1) * grads[0]
    m_b = beta1 * m_b + (1 - beta1) * grads[1]    
    # 二阶动量表达式和RMSProp的二阶动量表达式是一样的
    v_w = beta2 * v_w + (1 - beta2) * tf.square(grads[0])
    v_b = beta2 * v_b + (1 - beta2) * tf.square(grads[1])
    
    m_w_correction = m_w / (1 - tf.pow(beta1,int(global_step)))
    m_b_correction = m_b / (1 - tf.pow(beta1,int(global_step)))
    v_w_correction = v_w / (1 - tf.pow(beta2,int(global_step)))
    v_b_correction = v_b / (1 - tf.pow(beta2,int(global_step)))
    
    # 把修正项带入参数更新公式
    w1.assign_sub(lr * m_w_correction / tf.sqrt(v_w_correction))
    b1.assign_sub(lr * m_b_correction / tf.sqrt(v_b_correction))